In [85]:
    import pandas as pd
    import numpy as np

In [86]:
netflix_overall = pd.read_csv('netflix_titles.csv')

In [87]:
netflix_data = pd.read_csv('netflix_titles.csv')

In [88]:
netflix_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


# Netflix Recommendation system

![title](https://miro.medium.com/max/828/1*1b-yMSGZ1HfxvHiJCiPV7Q.png)

In [89]:
netflix_data = netflix_data.fillna('')

In [90]:
def clean_data(x):
        return str.lower(x.replace(" ", ""))

In [91]:
new_features = ['title', 'director', 'cast', 'listed_in', 'description']
netflix_data = netflix_data[new_features]

In [92]:
for new_features in new_features:
    netflix_data[new_features] = netflix_data[new_features].apply(clean_data)

In [93]:
netflix_data.head()

,title,director,cast,listed_in,description
0,normofthenorth:kingsizedadventure,"richardfinn,timmaltby","alanmarriott,andrewtoth,briandobson,colehoward...","children&familymovies,comedies",beforeplanninganawesomeweddingforhisgrandfathe...
1,jandino:whateverittakes,,jandinoasporaat,stand-upcomedy,jandinoasporaatriffsonthechallengesofraisingki...
2,transformersprime,,"petercullen,sumaleemontano,frankwelker,jeffrey...",kids'tv,"withthehelpofthreehumanallies,theautobotsoncea..."
3,transformers:robotsindisguise,,"willfriedle,darrencriss,constancezimmer,kharyp...",kids'tv,whenaprisonshipcrashunleasheshundredsofdecepti...
4,#realityhigh,fernandolebrija,"nestacooper,katewalsh,johnmichaelhiggins,keith...",comedies,whennerdyhighschoolerdanifinallyattractstheint...


In [94]:
def create_soup(x):
    return x['title']+ ' ' + x['director'] + ' ' + x['cast'] + ' ' +x['listed_in']+' '+ x['description']

In [95]:
netflix_data['soup'] = netflix_data.apply(create_soup, axis=1)


In [96]:
netflix_data.head(1)

,title,director,cast,listed_in,description,soup
0,normofthenorth:kingsizedadventure,"richardfinn,timmaltby","alanmarriott,andrewtoth,briandobson,colehoward...","children&familymovies,comedies",beforeplanninganawesomeweddingforhisgrandfathe...,"normofthenorth:kingsizedadventure richardfinn,..."


In [128]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(netflix_data['soup'])
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [129]:
netflix_data=netflix_data.reset_index()

In [130]:
indices = pd.Series(netflix_data.index, index=netflix_data['title'])

In [131]:
indices.head()

title
normofthenorth:kingsizedadventure    0
jandino:whateverittakes              1
transformersprime                    2
transformers:robotsindisguise        3
#realityhigh                         4
dtype: int64

In [132]:
result = 0
def get_recommendations_new(title, cosine_sim= cosine_sim2):
    global result
    
    title=title.replace(' ','').lower()
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    result =  netflix_overall['title'].iloc[movie_indices]
    return result

In [133]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    Value = "Enter Movie/TV-Show name",
    description = "Movie Title:",
    diabled = False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) >5:
            display(get_recommendations_new(title))

movie_input.observe(on_type, names='value')

display(movie_input, movie_list)

Text(value='', description='Movie Title:')

Output()

In [122]:
x=[]
for i in netflix_overall.title :
    x.append(i)

In [123]:
global p, result,test
def get_recommendations_new(titles,cosine_sim= cosine_sim2):
        
        try:
            titles=titles.replace(' ','').lower()
            idx = indices[titles] #from this we well get the index of that title in the data set.
             
            
            #age_rating = netflix_data['rating'][idx] #from this we get movie rating for age restriction
            #age_req = age_dict[age_rating] 

            #if(age_req <= user_age): # comparies user age to required age to watch movie
                
                # Get the pairwsie similarity scores of all movies with that movie
            sim_scores = list(enumerate(cosine_sim[idx]))

            # Sort the movies based on the similarity scores
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            # Get the scores of the 10 most similar movies
            sim_scores = sim_scores[1:11]

            # Get the movie indices
            movie_indices = [i[0] for i in sim_scores]
                # Return the top 10 most similar movies
            print("These are Recommended movies based upon your last watch")
            p = netflix_data['title'].iloc[movie_indices]
            #else:
            #    p = print('Your age should be: ',age_req,' but your age is: ', user_age," try with other movies")
            t = type(p)
            print(t)
            print('this is try loop')
            del p['index']
            test = 0
            return  p
        
        except KeyError:
            result = []
            words = [titles]
            global i
            i = 0
            print('this is except loop')
            test = 1
            def check_all(sentence, ws):
                return all(w in sentence for w in ws)

            for sentence in x:
                if any(check_all(sentence.lower(), word.split(' ')) for word in words):
                    print(sentence)
                    result.append(sentence)
                    i = 1
            result = pd.DataFrame (result, columns = ['Titles'])
            if i>0:
                print("\n","your title matches to these movie name, you can try these suggestions\n")
            else:
                print("entered movie is not at updated or Spelling mistake happened")
            t = type(result)
            print(t)
            return result

In [115]:
def get_recommendations_new(titles,cosine_sim= cosine_sim2):
    global result,j


    try:
        titles=titles.replace(' ','').lower()
        idx = indices[titles] #from this we well get the index of that title in the data set.


            #age_rating = netflix_data['rating'][idx] #from this we get movie rating for age restriction
            #age_req = age_dict[age_rating]

            #if(age_req <= user_age): # comparies user age to required age to watch movie

                # Get the pairwsie similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        print('this is try loop')
            # Sort the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            # Get the scores of the 10 most similar movies
        sim_scores = sim_scores[1:11]

            # Get the movie indices
        movie_indices = [i[0] for i in sim_scores]
                # Return the top 10 most similar movies
        print("These are Recommended movies based upon your last watch")
        result = netflix_data['title'].iloc[movie_indices]
            #else:
            #    p = print('Your age should be: ',age_req,' but your age is: ', user_age," try with other movies")
        del result['index']
        j = 0
        return  result

    except KeyError:
        result = []
        words = [titles]
        print('tjis is except loop')
        global i
        i = 0
        def check_all(sentence, ws):
            return all(w in sentence for w in ws)
        for sentence in x:
            if any(check_all(sentence.lower(), word.split(' ')) for word in words):
                print(sentence)
                result.append(sentence)
                i = 1
        result = pd.DataFrame (result, columns = ['Titles'])
        if i>0:
            print("\n","your title matches to these movie name, you can try these suggestions\n")
        else:
            print("entered movie is not at updated or Spelling mistake happened")
        j = 1
        return result

In [152]:
get_recommendations_new('PK', cosine_sim2)


5054                           3 Idiots
5494       The Legend of Michael Mishra
3093                  Anthony Kaun Hai?
2196                             Haapus
691                               Sanju
4110                   Taare Zameen Par
1449                    Chance Pe Dance
2194                    Chal Dhar Pakad
1746    EMI: Liya Hai To Chukana Padega
4920                   Khosla Ka Ghosla
Name: title, dtype: object

In [125]:
get_recommendations_new('tru', cosine_sim2)

this is except loop
True: Tricky Treat Day
Truth or Dare
Rodney Carrington: Here Comes the Truth
Meet the Trumps: From Immigrant to President
Monty Python's Almost the Truth
True: Winter Wishes
True Memoirs of an International Assassin
True: Grabbleapple Harvest
The Truth
LoveTrue
Dance & Sing with True
In Family I Trust
Trump: An American Dream
Buddy Thunderstruck
True to the Game
EVANGELION: DEATH (TRUE)²
True: Magical Friends
True: Wonderful Wishes
Mr. Gaga: A True Story of Love and Dance
My True Friend
Jimmy: The True Story of a True Idiot
Buddy Thunderstruck: The Maybe Pile
MegaTruckers
True Tunes
The Truth Seekers
Agatha and the Truth of Murder
Shadow of Truth
Twisted Trunk, Big Fat Body
Instructions Not Included
True Grit
Await Further Instructions
Truckbhar Swapna
True: Happy Hearts Day
Highly Strung
In Family We Trust
Avicii: True Stories
Struggle: The Life and Lost Art of Szukalski
The Truth About Alcohol
The Ugly Truth
The Trust
Outback Truckers
True and the Rainbow Kingdom


,Titles
0,True: Tricky Treat Day
1,Truth or Dare
2,Rodney Carrington: Here Comes the Truth
3,Meet the Trumps: From Immigrant to President
4,Monty Python's Almost the Truth
5,True: Winter Wishes
6,True Memoirs of an International Assassin
7,True: Grabbleapple Harvest
8,The Truth
9,LoveTrue


In [126]:
get_recommendations_new('iron', cosine_sim2)


this is except loop
Kabaneri of the Iron Fortress: The Battle of Unato
Iron Man & Captain America: Heroes United
Generation Iron 2
Iron Sky: The Coming Race
The Iron Lady
Iron Cowboy: The Story of the 50.50.50
Iron Ladies
Marvel's Iron Man & Hulk: Heroes United
Iron Fists and Kung-Fu Kicks
The Iron Lady
Generation Iron 3
Marvel's Iron Fist
Mobile Suit Gundam: Iron-Blooded Orphans

 your title matches to these movie name, you can try these suggestions

<class 'pandas.core.frame.DataFrame'>


,Titles
0,Kabaneri of the Iron Fortress: The Battle of U...
1,Iron Man & Captain America: Heroes United
2,Generation Iron 2
3,Iron Sky: The Coming Race
4,The Iron Lady
5,Iron Cowboy: The Story of the 50.50.50
6,Iron Ladies
7,Marvel's Iron Man & Hulk: Heroes United
8,Iron Fists and Kung-Fu Kicks
9,The Iron Lady


In [48]:
movies = pd.read_csv('netflix_titles.csv')
movies.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."


In [107]:
import re

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)
movies = netflix_data

In [108]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [109]:
movies.head(3)

,index,title,director,cast,listed_in,description,soup,clean_title
0,0,normofthenorth:kingsizedadventure,"richardfinn,timmaltby","alanmarriott,andrewtoth,briandobson,colehoward...","children&familymovies,comedies",beforeplanninganawesomeweddingforhisgrandfathe...,"normofthenorth:kingsizedadventure richardfinn,...",normofthenorthkingsizedadventure
1,1,jandino:whateverittakes,,jandinoasporaat,stand-upcomedy,jandinoasporaatriffsonthechallengesofraisingki...,jandino:whateverittakes jandinoasporaat stand...,jandinowhateverittakes
2,2,transformersprime,,"petercullen,sumaleemontano,frankwelker,jeffrey...",kids'tv,"withthehelpofthreehumanallies,theautobotsoncea...","transformersprime petercullen,sumaleemontano,...",transformersprime


In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [137]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = "Naruto"
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity  = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices][::-1]
    return results
#results

In [151]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    Value = "Enter Movie/TV-Show name",
    description = "Movie Title:",
    diabled = False
)
movie_input

Text(value='', description='Movie Title:')

In [146]:
movie_input = widgets.Text(
    Value = "Enter Movie/TV-Show name",
    description = "Movie Title:",
    diabled = False
    movie_input_2 = widgets.Text(
        Value = "Enter Age",
        description = "User Age",
        diabled = False
)
)


movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) >5:
            display(get_recommendations_new(title))

movie_input.observe(on_type, names='value_1', names= 'value_2')

display(movie_input, movie_list)

Text(value='')

In [149]:
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) >5:
            display(get_recommendations_new(title))

movie_input.observe(on_type, names='value_1', names= 'value_2')

display(movie_input, movie_list)

SyntaxError: keyword argument repeated: names (Temp/ipykernel_16020/3780641536.py, line 10)